In [1]:
import math
import uni_mathfunc as unim

<img src="img/pricetick1.png" width="400" height="340"> | <img src="img/pricetick2.png" width="400" height="340">


In [2]:
print(f"current price: {unim.price(1, 5000)}")
print(f"lower price: {unim.price(1, 4545)}")
print(f"upper price: {unim.price(1, 5500)}")

current price: 70.71067811865476
lower price: 67.4166151627327
upper price: 74.16198487095663


In [3]:
print(f"current tick: {unim.price_to_tick(unim.price(1,5000))}")
print(f"lower tick: {unim.price_to_tick(unim.price(1,4545))}")
print(f"upper tick: {unim.price_to_tick(unim.price(1,5500))}")

current tick: 85176
lower tick: 84222
upper tick: 86129


In [4]:
print(
    f"price USDC/ETH::5000/1 stored on smart contract: {unim.price_on_contract(unim.price(1, 5000))}"
)
print(
    f"price USDC/ETH::4545/1 stored on smart contract: {unim.price_on_contract(unim.price(1, 4545))}"
)
print(
    f"price USDC/ETH::5500/1 stored on smart contract: {unim.price_on_contract(unim.price(1, 5500))}"
)

price USDC/ETH::5000/1 stored on smart contract: 5.602277097478614e+30
price USDC/ETH::4545/1 stored on smart contract: 5.341294542274603e+30
price USDC/ETH::5500/1 stored on smart contract: 5.875717789736565e+30


概念：<span style="color:cyan">藍色線段</span> 注入流動性，C 點會往上推，代表 <span style="color:cyan">∆Y</span> 增加，所以 <span style="color:cyan">Y 幣</span> 增加流動性

概念：<span style="color:#8FBC8F">綠色線段</span> 注入流動性，C 點會往下推，代表 <span style="color:#8FBC8F">∆X</span> 增加，所以 <span style="color:#8FBC8F">X 幣</span> 增加流動性


<img src="img/liquidity.png" width="400" height="340"> | <img src="img/liquidity2.png" width="400" height="340">


## 個人解讀：

a 點 --> X 幣的最低價 (Y 幣最高價) ||| b 點 --> X 幣的最高價（Y 幣最低價）

在藍色線段上交易 --> 池子只需提供 Y 幣 ||| 在綠色線段交易 --> 池子只需提供 X 幣

當提供流動性時，我們希望價格穩定在 current price，所以要計算在藍綠兩個線段的流動性，並選擇較小的加入池子


<img src="img/calliquid1.png" width="400" height="350"> | <img src="img/calliquid3.png" width="400" height="350">


In [5]:
ether = 10**18

In [6]:
liq_x = unim.calc_liq_x(1 * ether, unim.price(1, 5500), unim.price(1,5000))
liq_x

1519437308014769733632

In [7]:
liq_y = unim.calc_liq_y(5000 * ether, unim.price(1, 4545), unim.price(1,5000))
liq_y

1517882343751509868544

In [8]:
# in fact we use the minimum liquidity when adding liuquidity to pool
min(liq_y, liq_x)

1517882343751509868544

<img src="img/liquidity2tokenAmount.png" width="400" height="350">

In [9]:
target_liquidity = min(liq_y, liq_x)
target_liquidity

1517882343751509868544

In [26]:
ETH_in_amount = unim.contract_recalc_deposit_amount_x(target_liquidity, unim.price(1, 5000), unim.price(1, 5500))
ETH_in_amount

998976618347425408

In [27]:
USDC_in_amount = unim.contract_recalc_deposit_amount_y(target_liquidity, unim.price(1, 5000), unim.price(1, 4545))
USDC_in_amount

5000000000000000000000

## Swapping


In [12]:
swap_in_USDC = 42 * ether

In [13]:
delta_p_USDC = swap_in_USDC / target_liquidity
delta_p_USDC

0.02767012882974529

In [14]:
new_p_USDC = (unim.price(1, 5000) + delta_p_USDC)
new_p_USDC

70.7383482474845

In [20]:
new_tick = unim.price_to_tick(new_p_USDC)
new_tick

85184

In [30]:
new_USDC_amount = unim.contract_recalc_deposit_amount_y(target_liquidity, unim.price(1,4545), new_p_USDC)
(new_USDC_amount - USDC_in_amount) /ether

41.999999999993186

In [23]:
new_ETH_amount = unim.contract_recalc_deposit_amount_x(target_liquidity, unim.price(1, 5500), new_p_USDC)
new_ETH_amount

990579904105264384

In [28]:
(new_ETH_amount - ETH_in_amount) / ether

-0.008396714242161023